# ArjanCode Tips

## ⚙️ Functions

[Video Url](https://www.youtube.com/watch?v=yatgY4NpZXE)

### 1. Do 1 thing and do it well

In [1]:
from dataclasses import dataclass
import datetime

In [2]:
@dataclass
class Customer:
    name: str
    phone: str
    cc_number: str
    cc_exp_month: int
    cc_exp_year: int
    cc_valid: bool = False

In [3]:
## OLD VERSION (Before refactoring)

def validate_card(customer: Customer) -> bool:
    def digitits_of(number: str) -> list[int]:
        return [int(d) for d in number]
    
    digits = digitits_of(customer.cc_number)
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = 0
    checksum += sum(odd_digits)
    for digit in even_digits:
        checksum += sum(digitits_of(str(digit * 2)))

    customer.cc_valid = (
        checksum % 10 == 0
        and datetime(customer.cc_exp_year, customer.cc_exp_month, 1) > datetime.now()
    )
    return customer.cc_valid

In [4]:
def main() -> None:
    alice = Customer(
        name="alice",
        phone="2341",
        cc_number="124",
        cc_exp_month=1,
        cc_exp_year=2024,
    )
    is_valid = validate_card(alice)
    print(f"Is Alice's card valid? {is_valid}")
    print(alice)

main()

Is Alice's card valid? False
Customer(name='alice', phone='2341', cc_number='124', cc_exp_month=1, cc_exp_year=2024, cc_valid=False)


In [5]:
## NEW VERSION 
def luhn_checksum(card_number: str) -> bool:
    def digitits_of(number: str) -> list[int]:
        return [int(d) for d in number]
    
    digits = digitits_of(card_number)
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = 0
    checksum += sum(odd_digits)
    for digit in even_digits:
        checksum += sum(digitits_of(str(digit * 2)))
    return checksum % 10 == 0

def validate_card(customer: Customer) -> bool:
    customer.cc_valid = (
        luhn_checksum(customer.cc_number)
        and datetime(customer.cc_exp_year, customer.cc_exp_month, 1) > datetime.now()
    )
    return customer.cc_valid

### 2. Separate commands from queries

In [6]:
def validate_card(customer: Customer) -> bool:
    return (
        luhn_checksum(customer.cc_number)
        and datetime(customer.cc_exp_year, customer.cc_exp_month, 1) > datetime.now()
    )

def main() -> None:
    alice = Customer(
        name="alice",
        phone="2341",
        cc_number="124",
        cc_exp_month=1,
        cc_exp_year=2024,
    )
    alice.cc_valid = validate_card(alice)
    print(f"Is Alice's card valid? {alice.cc_valid}")
    print(alice)

main()

Is Alice's card valid? False
Customer(name='alice', phone='2341', cc_number='124', cc_exp_month=1, cc_exp_year=2024, cc_valid=False)


### 3. Only request information you actually need

In [10]:
# validate_card was requesting the full Customer, but was not required all the Customer parameters
# "*" forces the use of keywords
def validate_card(*, number: int, exp_year: int, exp_month: int) -> bool:
    return (
        luhn_checksum(number)
        and datetime(exp_year, exp_month, 1) > datetime.now()
    )

def main() -> None:
    alice = Customer(
        name="alice",
        phone="2341",
        cc_number="124",
        cc_exp_month=1,
        cc_exp_year=2024,
    )
    alice.cc_valid = validate_card(
        number = alice.cc_number, 
        exp_month = alice.cc_exp_year, 
        exp_year = alice.cc_exp_month
    )
    
    print(f"Is Alice's card valid? {alice.cc_valid}")
    print(alice)

main()

Is Alice's card valid? False
Customer(name='alice', phone='2341', cc_number='124', cc_exp_month=1, cc_exp_year=2024, cc_valid=False)


### 4. Keep the number of parameters minimal

In [27]:
# the validate_card has too many parameters -> validate_card(*, number: int, exp_year: int, exp_month: int)
# if too many parameters, maybe the functions is trying to do too many things instead of doing a single task
# improvements:
## 1. Could add default values (Sometimes, not possible example exp_year)
## 2. Solution: Introduce abstraction by the use of the class CardInfo


from typing import Protocol

class CardInfo(Protocol):
    @property
    def cc_number(self) -> str:
        ...

    @property
    def cc_exp_year(self) -> str:
        ...

    @property
    def cc_exp_month(self) -> str:
        ...


# new version of validate_card only requires 1 parameter
def validate_card(card: CardInfo) -> bool:
    return (
        luhn_checksum(card.cc_number)
        and datetime(card.cc_exp_year, card.cc_exp_month, 1) > datetime.now()
    )

def main() -> None:
    alice = Customer(
        name="alice",
        phone="2341",
        cc_number="124",
        cc_exp_month=1,
        cc_exp_year=2024,
    )
    alice.cc_valid = validate_card(alice) # here we only need to call with the customer
    
    print(f"Is Alice's card valid? {alice.cc_valid}")
    print(alice)

main()

# validate_card doesn't need to know nothing about the Customer it's juts needs an object
# that has a cc_number, cc_exp_year and cc_exp_month

Is Alice's card valid? False
Customer(name='alice', phone='2341', cc_number='124', cc_exp_month=1, cc_exp_year=2024, cc_valid=False)


In [32]:
# Still for now the class Customer contains too many variables and has the potential to become a big class
# Solution: introduce classes to represent sub-objects

@dataclass
class Card: 
    number: str
    exp_month: int
    exp_year: int

@dataclass
class Customer:
    name: str
    phone: str
    card: Card
    cc_valid: bool = False

# just rename the variables
def validate_card(card: CardInfo) -> bool:
    return (
        luhn_checksum(card.number)
        and datetime(card.exp_year, card.exp_month, 1) > datetime.now()
    )

def main() -> None:
    card = Card(number="124", exp_month=1, exp_year=2024)
    alice = Customer(name="alice", phone="2341", card=card)
    alice.cc_valid = validate_card(card) 
    
    print(f"Is Alice's card valid? {alice.cc_valid}")
    print(alice)

main()

# Parameters and Arguments are not the same!!!
# 
# Parameters are part of the defenition of a function
# Agruments are the values that you set to this parameters 

Is Alice's card valid? False
Customer(name='alice', phone='2341', card=Card(number='124', exp_month=1, exp_year=2024), cc_valid=False)


### 5. Don't create and use an object in the same place

In [45]:
### BEFORE
import logging

class StripePaymentHandler:
    def handle_payment(self, amount: int) -> None:
        logging.info(f"Charging {amount/100:.2f} € using Stripe")

PRICES = {
    "burger": 10_00,
    "fries": 5_00,
    "drink": 2_00,
    "salad": 15_00,
}

def order_food(items: list[str]) -> None:
    total = sum(PRICES[item] for item in items)
    logging.info(f"Order total is {total/100:.2f} €.")
    payment_handler = StripePaymentHandler()
    payment_handler.handle_payment(total)
    logging.info("Order completed.")

def main() -> None:
    logging.basicConfig(level=logging.INFO, force=True)
    order_food(["burger", "fries", "drink"])


main()

INFO:root:Order total is 17.00 €.
INFO:root:Charging 17.00 € using Stripe
INFO:root:Order completed.


In [46]:
#### AFTER

# payment_handler = StripePaymentHandler()
# payment_handler.handle_payment(total)
## This makes order_food harder to test becaus we need to patch the StripePaymentHandler
## INSTEAD we should create a dependeny injection
def order_food(items: list[str], payment_handler:StripePaymentHandler ) -> None:
    total = sum(PRICES[item] for item in items)
    logging.info(f"Order total is {total/100:.2f} €.")
    payment_handler.handle_payment(total)
    logging.info("Order completed.")

def main() -> None:
    logging.basicConfig(level=logging.INFO, force=True)
    payment_handler = StripePaymentHandler()
    order_food(["burger", "fries", "drink"], payment_handler)


main()

INFO:root:Order total is 17.00 €.
INFO:root:Charging 17.00 € using Stripe
INFO:root:Order completed.


In [61]:
#### Even better using abstraction

from typing import Protocol

class StripePaymentHandler:
    def handle_payment(self, amount: int) -> None:
        logging.info("Charging  %.2f € using Stripe", amount/100)

class PaymentHandler(Protocol):
    @property
    def handle_payment(self, amount: int) -> None:
        ...

def order_food(items: list[str], payment_handler: PaymentHandler) -> None:
    total = sum(PRICES[item] for item in items)
    logging.info("Order total is %.2f € in %s in %s", total/100, "Portugal", "Porto")
    payment_handler.handle_payment(total)
    logging.info("Order completed.")

def main() -> None:
    logging.basicConfig(level=logging.INFO, force=True)
    payment_handler = StripePaymentHandler()
    order_food(["burger", "fries", "drink"], payment_handler)


main()

INFO:root:Order total is 17.00 € in Portugal in Porto
INFO:root:Charging  17.00 € using Stripe
INFO:root:Order completed.


### 6. Don't use flag arguments

In [5]:
from dataclasses import dataclass
from enum import StrEnum, auto

FIXED_VACATION_DAYS_PAYOUT = 5

class Role(StrEnum):
    PRESIDENT = auto()
    VICEPRESIDENT = auto()
    MANAGER = auto()
    LEAD = auto()
    ENGINEER = auto()
    INTERN = auto()

@dataclass
class Employee:
    name: str
    role: Role
    vacation_days: int = 25

    def take_a_holiday(self, payout: bool, nr_days: int = 1) -> None:
        if payout:
            if self.vacation_days < FIXED_VACATION_DAYS_PAYOUT:
                raise ValueError(
                    f"You don't have enough holidays left over for a payout.\
                        Remaining holidays:{self.vacation_days} ")
            self.vacation_days -= FIXED_VACATION_DAYS_PAYOUT
            print(f"Paying out a holiday. Holidays left: {self.vacation_days}")
        else:
            if  self.vacation_days < nr_days:
                raise ValueError(
                    "You don't have any holidays left. Now back to work, you!"
                )
            self.vacation_days -= nr_days
            print("Have fun on your holiday. Don't forget to check your emails!")

def main() -> None:
    employee = Employee(name="John Doe", role=Role.ENGINEER)
    employee.take_a_holiday(True)

if __name__ == "__main__":
    main()

Paying out a holiday. Holidays left: 20


In [7]:
## AFTER
## With the methods split in 2 we no loger need a bollean flag and also there's no need to input parameters hat
# # otherwise wouldn't make sense 
@dataclass
class Employee:
    name: str
    role: Role
    vacation_days: int = 25

    def payout_holiday(self) -> None:
        if self.vacation_days < FIXED_VACATION_DAYS_PAYOUT:
            raise ValueError(
                f"You don't have enough holidays left over for a payout.\
                    Remaining holidays:{self.vacation_days} ")
        self.vacation_days -= FIXED_VACATION_DAYS_PAYOUT
        print(f"Paying out a holiday. Holidays left: {self.vacation_days}")

    def take_holyday(self, nr_days: int = 1) -> None:
        if  self.vacation_days < nr_days:
            raise ValueError(
                "You don't have any holidays left. Now back to work, you!"
            )
        self.vacation_days -= nr_days
        print("Have fun on your holiday. Don't forget to check your emails!")

def main() -> None:
    employee = Employee(name="John Doe", role=Role.ENGINEER)
    employee.take_holyday(True)

if __name__ == "__main__":
    main()

Have fun on your holiday. Don't forget to check your emails!


### 7. Remeber that functions are objects

In [10]:
#### Before
import logging
from typing import Protocol

PRICES = {
    "burger": 10_00,
    "fries": 5_00,
    "drink": 2_00,
    "salad": 15_00,
}

class StripePaymentHandler:
    def handle_payment(self, amount: int) -> None:
        logging.info("Charging  %.2f € using Stripe", amount/100)

class PaymentHandler(Protocol):
    @property
    def handle_payment(self, amount: int) -> None:
        ...

def order_food(items: list[str], payment_handler: PaymentHandler) -> None:
    total = sum(PRICES[item] for item in items)
    logging.info("Order total is %.2f € in %s in %s", total/100, "Portugal", "Porto")
    payment_handler.handle_payment(total)
    logging.info("Order completed.")

def main() -> None:
    logging.basicConfig(level=logging.INFO, force=True)
    payment_handler = StripePaymentHandler()
    order_food(["burger", "fries", "drink"], payment_handler)


main()

INFO:root:Order total is 17.00 € in Portugal in Porto
INFO:root:Charging  17.00 € using Stripe
INFO:root:Order completed.


In [13]:
#### AFTER
import logging
from typing import Callable

PRICES = {
    "burger": 10_00,
    "fries": 5_00,
    "drink": 2_00,
    "salad": 15_00,
}

def handle_stripe_payment(amount: int) -> None:
    logging.info("Charging  %.2f € using Stripe", amount/100)

HandlePaymentFn = Callable[[int], None]

def order_food(items: list[str], payment_handler: HandlePaymentFn) -> None:
    total = sum(PRICES[item] for item in items)
    logging.info("Order total is %.2f € in %s in %s", total/100, "Portugal", "Porto")
    payment_handler(total)
    logging.info("Order completed.")

def main() -> None:
    logging.basicConfig(level=logging.INFO, force=True)
    order_food(["burger", "fries", "drink"], handle_stripe_payment)


main()

INFO:root:Order total is 17.00 € in Portugal in Porto


AttributeError: 'function' object has no attribute 'handle_payment'